In [1]:
import os
import json
import boto3
import pandas as pd

In [2]:
buckets = ['lax-drawing-nuts-bolts-all', 'lax-drawing-furniture-all', 'lax-drawing-wheels-all']

In [8]:
def manifest_to_df(manifest):
    splits = ['train', 'test']
    metadata = manifest['metadata']
    base_name = metadata['name'][:-4]
    s3_paths = manifest['s3_stimuli_paths_to_local_name']
    rows = []

    for split in splits:
        mfst_split = manifest[split]
        condition = list(mfst_split.keys())[0]
        curriculum = list(mfst_split[condition].keys())[0]
        images = mfst_split[condition][curriculum]

    for image in s3_paths:
        stim_id = image.split('_')[-1][:-4]
        split = s3_paths[image].split('_')[-2]
        row = {'data_split': split, 'base_name': base_name, 's3_path': image, 'stim_id': stim_id, 'curriculum': curriculum, 'condition': condition, 'metadata': metadata}
        rows.append(row)

    df = pd.DataFrame(rows)
    return df

def upload_to_s3(manifest, bucket_name):
    s3 = boto3.resource("s3")
    s3.Object(bucket_name, 'manifest.csv').put(Body=manifest)
    s3.Object(bucket_name, 'manifest.csv').Acl().put(ACL="public-read")

In [11]:
def main():
    for bucket in buckets:
        # Download manifests
        s3 = boto3.client('s3')
        s3.download_file(bucket, 'manifest.json', bucket + "-manifest.json")
        
        with open(bucket + "-manifest.json", "rb") as f:
                manifest = json.load(f)
        
        # convert to CSV and reupload
        df = manifest_to_df(manifest)
        df.to_csv(bucket + "-manifest.csv")
        
        with open(bucket + "-manifest.csv", "r") as f:
            
        print(csv_manifest)
        upload_to_s3(csv_manifest, bucket)
        print(f"Success! Uploaded new manifest to {bucket}")

In [12]:
main()

None


ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value: None, type: <class 'NoneType'>, valid types: <class 'bytes'>, <class 'bytearray'>, file-like object